# Glacier evolution postprocessing

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import os
from glob import glob
from tqdm import tqdm
from tqdm.notebook import tqdm

variables = ["melt_off_glacier_daily", "melt_on_glacier_daily", "liq_prcp_off_glacier_daily", "liq_prcp_on_glacier_daily"]

In [ ]:
# useful functions
def julian_to_gregorian(year, julian_day):

    year = int(year)
    julian_day = int(julian_day)
    gregorian_date = pd.to_datetime(f'{int(year)}-01-01')
    gregorian_date += pd.to_timedelta(julian_day - 1, unit='D')
    gregorian_date = np.array(gregorian_date, dtype='datetime64[ns]')
    return gregorian_date

def change_calendar(ds):

    ds = ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month', 'calendar_day_2d'])
    ds = ds.stack(datetime=("time", "day_2d"))
    dates = [julian_to_gregorian(year, day) for year, day in ds.datetime.values]
    ds = ds.drop_vars(["datetime", 'time', 'day_2d'])
    ds["datetime"] = dates
    ds = ds.dropna("datetime", how = "all")
    return ds

def hydro_variables(ds): # calculate total_runoff
    ds["glacier_runoff"] = ((ds.melt_off_glacier_daily + ds.melt_on_glacier_daily + ds.liq_prcp_off_glacier_daily + ds.liq_prcp_on_glacier_daily)*1e-3)/(86400) # m3/s
    return ds["glacier_runoff"]

## Data

In [ ]:
# paths
path_raw   = "/home/rooda/Hydro_results/oggm_runs_daily/"
path_final = "/home/rooda/OneDrive/DeepHydro/"
path_pmet  = "/home/rooda/OneDrive/PatagoniaMet/"

# glacier dataset
RGI_ids    = gpd.read_file(path_final + "GIS/Glaciers/RGI7_Hydro.shp")

# catchment dataset
q_metadata   = pd.read_csv(path_pmet + "Zenodo/v11/Q_PMETobs_v11_metadata.csv", index_col = "gauge_id")
q_metadata["gauge_code"] = q_metadata["gauge_code"].astype("str")

## Historical period

### Glacier runoff

In [ ]:
# for all patagonian catchments
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc")[variables]
oggm_ts  = oggm_ts.sel(time = slice(1990,2019)) # 2020 with no data
oggm_ts  = change_calendar(oggm_ts)
oggm_ts  = hydro_variables(oggm_ts)
oggm_ts  = oggm_ts.assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
oggm_ts  = oggm_ts.groupby('rgi_id').sum()
oggm_ts  = oggm_ts.to_dataframe().reset_index().rename(columns = {"datetime": "time"})
oggm_ts  = oggm_ts.pivot(index = "time", columns = "rgi_id", values = "total_runoff")
oggm_ts  = oggm_ts.round(3) # save some space
oggm_ts.to_csv(path_final + "Runoff/glacier_runoff_historical_all.csv")

In [ ]:
# for PMET-obs catchments (calibration)
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc")[variables]
oggm_ts  = oggm_ts.sel(time = slice(1980,2019)) # 2020 with no data
oggm_ts  = change_calendar(oggm_ts)
oggm_ts  = hydro_variables(oggm_ts)

oggm_ts_ds = pd.DataFrame(columns = q_metadata.index, index = oggm_ts.datetime.values)
for basin in tqdm(q_metadata.index):

    # candidates
    id_candidates = (q_metadata['gauge_code'].str.startswith(q_metadata['gauge_code'].loc[basin]) & 
                   (q_metadata['gauge_code'].str.len() >= len(q_metadata['gauge_code'].loc[basin])))
    id_candidates = q_metadata['gauge_code'][id_candidates]
    
    if np.sum(RGI_ids['ID_PMET'].isin(id_candidates)) > 0:

        rgi_candidates = RGI_ids[RGI_ids.ID_PMET.isin(id_candidates)].RGIId
        oggm_ts_basin = oggm_ts.sel(rgi_id = rgi_candidates.to_list())
        oggm_ts_basin = oggm_ts_basin.sum(dim = "rgi_id").to_series()
        oggm_ts_ds[basin] = oggm_ts_basin

oggm_ts_ds = oggm_ts_ds.fillna(0).round(3) 
oggm_ts_ds.index.name = "time"
oggm_ts_ds.to_csv(path_final + "Runoff/glacier_runoff_historical_pmet.csv")

### Glacier melt

In [ ]:
# for all patagonian catchments
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc")[variables]
oggm_ts  = oggm_ts.sel(time = slice(1990,2019)) # 2020 with no data
oggm_ts  = change_calendar(oggm_ts)["melt_on_glacier_daily"]
oggm_ts  = oggm_ts*1e-3/86400
oggm_ts  = oggm_ts.assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
oggm_ts  = oggm_ts.groupby('rgi_id').sum()
oggm_ts  = oggm_ts.to_dataframe().reset_index().rename(columns = {"datetime": "time"})
oggm_ts  = oggm_ts.pivot(index = "time", columns = "rgi_id", values = "melt_on_glacier_daily")
oggm_ts  = oggm_ts.round(3) # save some space
oggm_ts.to_csv(path_final + "Runoff/glacier_melt_historical_all.csv")

In [ ]:
# for PMET-obs catchments (calibration)
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc")[variables]
oggm_ts  = oggm_ts.sel(time = slice(1980,2019)) # 2020 with no data
oggm_ts  = change_calendar(oggm_ts)["melt_on_glacier_daily"]
oggm_ts  = oggm_ts*1e-3/86400

oggm_ts_ds = pd.DataFrame(columns = q_metadata.index, index = oggm_ts.datetime.values)
for basin in tqdm(q_metadata.index):

    # candidates
    id_candidates = (q_metadata['gauge_code'].str.startswith(q_metadata['gauge_code'].loc[basin]) & 
                   (q_metadata['gauge_code'].str.len() >= len(q_metadata['gauge_code'].loc[basin])))
    id_candidates = q_metadata['gauge_code'][id_candidates]
    
    if np.sum(RGI_ids['ID_PMET'].isin(id_candidates)) > 0:

        rgi_candidates = RGI_ids[RGI_ids.ID_PMET.isin(id_candidates)].RGIId
        oggm_ts_basin = oggm_ts.sel(rgi_id = rgi_candidates.to_list())
        oggm_ts_basin = oggm_ts_basin.sum(dim = "rgi_id").to_series()
        oggm_ts_ds[basin] = oggm_ts_basin

oggm_ts_ds = oggm_ts_ds.fillna(0).round(3) 
oggm_ts_ds.index.name = "time"
oggm_ts_ds.to_csv(path_final + "Runoff/glacier_melt_historical_pmet.csv")

### Volume

In [ ]:
# for all patagonian catchments
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc").volume
oggm_ts = oggm_ts.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])
oggm_ts  = oggm_ts.assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
oggm_ts  = oggm_ts.groupby('rgi_id').sum()
oggm_ts  = oggm_ts.to_dataframe().reset_index().rename(columns = {"datetime": "time"})
oggm_ts  = oggm_ts.pivot(index = "time", columns = "rgi_id", values = "volume")
oggm_ts  = oggm_ts.round(3) # save some space
oggm_ts.to_csv(path_final + "Runoff/glacier_volume_historical_all.csv")

In [ ]:
# for PMET-obs catchments (calibration)
oggm_ts  = xr.open_dataset(path_raw + "run_output_historical.nc").volume
oggm_ts = oggm_ts.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])

oggm_ts_ds = pd.DataFrame(columns = q_metadata.index, index = oggm_ts.time.values)
for basin in tqdm(q_metadata.index):

    # candidates
    id_candidates = (q_metadata['gauge_code'].str.startswith(q_metadata['gauge_code'].loc[basin]) & 
                   (q_metadata['gauge_code'].str.len() >= len(q_metadata['gauge_code'].loc[basin])))
    id_candidates = q_metadata['gauge_code'][id_candidates]
    
    if np.sum(RGI_ids['ID_PMET'].isin(id_candidates)) > 0:

        rgi_candidates = RGI_ids[RGI_ids.ID_PMET.isin(id_candidates)].RGIId
        oggm_ts_basin = oggm_ts.sel(rgi_id = rgi_candidates.to_list())
        oggm_ts_basin = oggm_ts_basin.sum(dim = "rgi_id").to_series()
        oggm_ts_ds[basin] = oggm_ts_basin

oggm_ts_ds = oggm_ts_ds.fillna(0).round(3) 
oggm_ts_ds.index.name = "time"
oggm_ts_ds.to_csv(path_final + "Runoff/glacier_volume_historical_pmet.csv")

## Future period

In [ ]:
gcm_list  = ["GFDL-ESM4", "IPSL-CM6A-LR", "MIROC6", "MPI-ESM1-2-LR", "MRI-ESM2-0"]
ssp_list  = ["ssp126", "ssp585"]

In [ ]:
# Glacier runoff (daily)

df_ssp = []
for ssp in tqdm(ssp_list):
    
    df_gcm = []
    for gcm in tqdm(gcm_list, leave = False): 
        oggm_ts  = xr.open_mfdataset(path_raw + "run_output_{}_{}*.nc".format(gcm,ssp), combine='nested', concat_dim= "rgi_id")[variables].load()
        oggm_ts  = change_calendar(oggm_ts)
        oggm_ts  = hydro_variables(oggm_ts)
        oggm_ts  = oggm_ts.sortby("rgi_id").assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
        oggm_ts  = oggm_ts.groupby('rgi_id').sum()
        df_gcm.append(oggm_ts)
    
    df_gcm = xr.concat(df_gcm, dim='gcm')
    df_ssp.append(df_gcm)

df_ssp = xr.concat(df_ssp, dim='ssp')
df_ssp = df_ssp.assign_coords(gcm=gcm_list, ssp=ssp_list)
df_ssp = df_ssp.rename({'datetime': 'time'})
df_ssp = df_ssp.sel(time = slice("2020-01-01","2099-12-31")) 
df_ssp = df_ssp.astype("float32")
df_ssp.to_netcdf(path_final + "Runoff/glacier_runoff_future_all.nc", 
                 encoding = {'glacier_runoff': {'dtype': 'int32', 'scale_factor': 0.001, '_FillValue': -9999, 'zlib': True, 'complevel': 1}})

In [ ]:
# Glacier melt (in m3/s)

df_ssp = []
for ssp in tqdm(ssp_list):
    
    df_gcm = []
    for gcm in tqdm(gcm_list, leave = False): 
        oggm_ts  = xr.open_mfdataset(path_raw + "run_output_{}_{}*.nc".format(gcm,ssp), combine='nested', concat_dim= "rgi_id").melt_on_glacier_daily.load()
        oggm_ts  = change_calendar(oggm_ts)
        oggm_ts  = oggm_ts*1e-3/86400
        oggm_ts  = oggm_ts.sortby("rgi_id").assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
        oggm_ts  = oggm_ts.groupby('rgi_id').sum()
        df_gcm.append(oggm_ts)
    
    df_gcm = xr.concat(df_gcm, dim='gcm')
    df_ssp.append(df_gcm)

df_ssp = xr.concat(df_ssp, dim='ssp')
df_ssp = df_ssp.assign_coords(gcm=gcm_list, ssp=ssp_list)
df_ssp = df_ssp.rename({'datetime': 'time'})
df_ssp = df_ssp.sel(time = slice("2020-01-01","2099-12-31")) 
df_ssp = df_ssp.astype("float32")
df_ssp.to_netcdf(path_final + "Runoff/glacier_melt_future_all.nc", 
                 encoding = {'melt_on_glacier_daily': {'dtype': 'int32', 'scale_factor': 0.001, '_FillValue': -9999, 'zlib': True, 'complevel': 1}})

In [ ]:
# Glacier volume (in m3)

df_ssp = []
for ssp in tqdm(ssp_list):
    
    df_gcm = []
    for gcm in tqdm(gcm_list, leave = False): 
        oggm_ts  = xr.open_mfdataset(path_raw + "run_output_{}_{}*.nc".format(gcm,ssp), combine='nested', concat_dim= "rgi_id")["volume"]
        oggm_ts  = oggm_ts.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month']).load()
        oggm_ts  = oggm_ts.sortby("rgi_id").assign_coords(rgi_id = RGI_ids.ID_basin.tolist())
        oggm_ts  = oggm_ts.groupby('rgi_id').sum()
        df_gcm.append(oggm_ts)
    
    df_gcm = xr.concat(df_gcm, dim='gcm')
    df_ssp.append(df_gcm)

df_ssp = xr.concat(df_ssp, dim='ssp')
df_ssp = df_ssp.assign_coords(gcm=gcm_list, ssp=ssp_list)
df_ssp = df_ssp.astype("int64")
df_ssp.to_netcdf(path_final + "Runoff/glacier_volume_future_all.nc", 
                 encoding = {"volume": {'dtype': 'int64', 'zlib': True, 'complevel': 1}})